In [ ]:
1+1

In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

import pid
from pyanalib import panda_helpers
import os
import multiprocessing

In [ ]:
from pyanalib import variable, histogram

In [ ]:
from pyanalib.dataset import Dataset

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)
importlib.reload(variable)

In [ ]:
mpl.rc('font', size=14)

dosave = False
savedir = "./plots_11_20_23/"

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
datadir = "/icarus/data/users/gputnam/calib-data/outputs/"

In [ ]:
filedir = "/icarus/data/users/gputnam/DMCP2023G/"

onbeamfile_Run1 = filedir + "data-reprod-D/dfs-wstubs/Run1OnBeamReprodD_evt.df"
onbeamfile_Run2 = filedir + "data-reprod-D/dfs-wstubs/Run2OnBeamReprodD_evt.df"

offbeamfile_Run1 = filedir + "data-reprod-D/dfs-wstubs/Run1OffBeamReprodD_evt.df"
offbeamfile_Run2 = filedir + "data-reprod-D/dfs-wstubs/Run2OffBeamReprodD_evt.df"

mcfile = "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/mcnuphase2_evt_reprodD.df"

mccohfile = "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/coh-like-evt-CV.df"

In [ ]:
mccoh_syst_files = [
    "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/coh-like-evt-ind1bin0.df",
    "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/coh-like-evt-ind1bin14.df",
    "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/coh-like-evt-ind0gainlo.df",
    "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/coh-like-evt-ind0gainhi.df",
    "/icarus/data/users/gputnam/DMCP2023G/mc/dfs-wstubs/coh-like-evt-noisehi.df",
]

syst_labels = [
    "Middle Ind. Opaque",
    "Middle Ind. Transparent",
    "Front Ind. Gain Low",
    "Front Ind. Gain High",
    "Noise 1.2x",
]

In [ ]:
@var.VAR
def cohlike(df):
    nmu = df.iscc & (np.abs(df.pdg) == 14)
    npi = df.npi
    ns = df.nsm + df.nsp

    is_coh_like = (nmu + npi + ns >= 2) & (df.max_proton_ke < 0.05)

    is_coh_like = is_coh_like & InFV(df.position, 50)

    return is_coh_like

In [ ]:
cohlike

In [ ]:
import gc

In [ ]:
gc.collect()

In [ ]:
mc_incoh = data.mc_dataset(mcfile, "evt", mccut=~cohlike, mccut_any=False)
mc_incoh.df

In [ ]:
mc_incoh.df["Run1"] = True
mc_incoh.df["Run2"] = True
mc_incoh.df["cohlike"] = False

In [ ]:
mccoh = data.mc_dataset(mccohfile, "evt")
mccoh.df

In [ ]:
mccoh.df["Run1"] = True
mccoh.df["Run2"] = True
mccoh.df["cohlike"] = True

In [ ]:
def concat(self, *othrs, offset=0, sum_pot=True):
    if sum_pot:
        POT = sum([o.POT for o in othrs]) + self.POT # POT is total
    else:
        POT = min([o.POT for o in othrs] + [self.POT])

    livetime = sum([o.livetime for o in othrs]) + self.livetime

    for i in range(len(othrs)):
        othrs[i].df.index = othrs[i].df.index.set_levels(othrs[i].df.index.levels[0] + offset*(i+1), level=0)
        if self.hdrdf is not None:
            othrs[i].hdrdf.index = othrs[i].hdrdf.index.set_levels(othrs[i].hdrdf.index.levels[0] + offset*(i+1), level=0)

    df = pd.concat([self.df] + [o.df for o in othrs])
    
    # redo weights if not summing the pot
    if not sum_pot:
        df.loc[self.df.index, ("wgt", "cv", )] = self.df.wgt.cv*(POT/self.POT)
        for i in range(len(othrs)):
            df.loc[othrs[i].df.index, ("wgt", "cv", )] = othrs[i].df.wgt.cv*(POT/othrs[i].POT)
    
    hdrdf = None
    if self.hdrdf is not None:
        hdrdf = pd.concat([self.hdrdf] + [o.hdrdf for o in othrs])

    return Dataset(df, livetime, POT, hdrdf)


In [ ]:
mc = concat(mc_incoh, mccoh, sum_pot=False)

In [ ]:
# Systematic variations
mc_systs_all = []
for i, f in enumerate(mccoh_syst_files):
    mccoh_syst = data.mc_dataset(f, "evt", syst_weights=False)
    mccoh_syst.df["cohlike"] = True
    mc_syst = concat(mc_incoh, mccoh_syst, sum_pot=False)
    mc_systs_all.append(mc_syst)

In [ ]:
mc_systs = [
    [mc_systs_all[0], mc_systs_all[1]], # vary middle ind
    [mc_systs_all[2], mc_systs_all[3]], # vary front ind
    [mc_systs_all[4]] # vary noise
    
]

In [ ]:
mccoh_syst_files

In [ ]:
onbeamR1 = data.onbeam_dataset(onbeamfile_Run1, "evt")
onbeamR2 = data.onbeam_dataset(onbeamfile_Run2, "evt")

In [ ]:
onbeamR1.df["Run1"] = True
onbeamR1.df["Run2"] = False

onbeamR2.df["Run1"] = False
onbeamR2.df["Run2"] = True

onbeam = onbeamR1.concat(onbeamR2)

In [ ]:
onbeamR1_half = data.onbeam_dataset(onbeamfile_Run1, "evt")
onbeamR2_half = data.onbeam_dataset(onbeamfile_Run2, "evt")

In [ ]:
onbeamR1_half.df = onbeamR1_half.df.iloc[::2]
onbeamR1_half.POT = onbeamR1_half.POT / 2
onbeamR1_half.livetime = onbeamR1_half.livetime / 2

onbeamR2_half.df = onbeamR2_half.df.iloc[::2]
onbeamR2_half.POT = onbeamR2_half.POT / 2
onbeamR2_half.livetime = onbeamR2_half.livetime / 2

onbeamR1_half.df["Run1"] = True
onbeamR1_half.df["Run2"] = False

onbeamR2_half.df["Run1"] = False
onbeamR2_half.df["Run2"] = True

onbeam_half = onbeamR1_half.concat(onbeamR2_half)

In [ ]:
offbeamR1 = data.offbeam_dataset(offbeamfile_Run1, "evt")
offbeamR2 = data.offbeam_dataset(offbeamfile_Run2, "evt")

In [ ]:
offbeamR1.df["Run1"] = True
offbeamR1.df["Run2"] = False

offbeamR2.df["Run1"] = False
offbeamR2.df["Run2"] = True

offbeam = offbeamR1.concat(offbeamR2)

In [ ]:
onbeam.df["cohlike"] = True
onbeam_half.df["cohlike"] = True
offbeam.df["cohlike"] = True

In [ ]:
POTSTR = "$%.2f \\times 10^{20}$ POT" % (onbeam.POT/1e20)
POTSTR

In [ ]:
POTSTR_half = "$%.2f \\times 10^{20}$ POT" % (onbeam_half.POT/1e20)
POTSTR_half

In [ ]:
cohlike = var.DF.cohlike

In [ ]:
nucategories = [
    ((var.slc.truth.iscc == 0) & (var.slc.tmatch.idx >= 0)), # NC
    ((var.slc.truth.pdg == 14) | (var.slc.truth.pdg == -14)) & (var.slc.truth.iscc==1) & ((var.slc.truth.genie_mode == 0) | (var.slc.truth.genie_mode == 10)), # CC QE+MEC
    ((var.slc.truth.pdg == 14) | (var.slc.truth.pdg == -14)) & (var.slc.truth.iscc==1) & (var.slc.truth.genie_mode == 1), # CC RES
    ((var.slc.truth.pdg == 14) | (var.slc.truth.pdg == -14)) & (var.slc.truth.iscc==1) & (var.slc.truth.genie_mode == 2), # CC DIS
    ((var.slc.truth.pdg == 14) | (var.slc.truth.pdg == -14)) & (var.slc.truth.iscc==1) & (var.slc.truth.genie_mode == 3), # CC COH
]
nucategories.insert(0, var.slc.tmatch.idx >= 0)
for c in nucategories[1:]:
    nucategories[0] = nucategories[0] & ~c
    
nucategories.insert(0, var.slc.tmatch.idx < 0)
    
nucategories[0].name = "Cosmic"
nucategories[1].name = "$\\nu$ Other"
nucategories[2].name = "$\\nu$ NC"
nucategories[3].name = "$\\nu_\\mu$ CC QE+MEC"
nucategories[4].name = "$\\nu_\\mu$ CC RES"
nucategories[5].name = "$\\nu_\\mu$ CC DIS"
nucategories[6].name = "$\\nu_\\mu$ CC COH"

nucategory_colors = [
    "lightgray",
    "#A57647",
    "#EAC387",
    "#97B9BB",
    "#C6C2D9",
    "#F0CBD2",
    "#983E49", 
]


In [ ]:
trunkIDcategories = [
    (var.DF.trunk.trk.truth.p.pdg == 13) | (var.DF.trunk.trk.truth.p.pdg == -13),
    (var.DF.trunk.trk.truth.p.pdg == 211) | (var.DF.trunk.trk.truth.p.pdg == -211),
    (var.DF.trunk.trk.truth.p.pdg == 2212),
    (var.DF.trunk.trk.truth.p.pdg == 22),
]
trunkIDcategories.append(~trunkIDcategories[0])
for c in trunkIDcategories[1:-1]:
    trunkIDcategories[-1] = trunkIDcategories[-1] & ~c
    
trunkIDcategories[0].name = "$\\mu$"
trunkIDcategories[1].name = "$\\pi$"
trunkIDcategories[2].name = "$p$"
trunkIDcategories[3].name = "$\\gamma$"
trunkIDcategories[4].name = "Other"

trunkIDcategory_colors = [
    "#07AC96",
    "#B076A8",
    "#DBAF2B",
    "#C61627",
    "#C3AA8F",
]

In [ ]:
branchIDcategories = [
    (var.DF.branch.trk.truth.p.pdg == 13) | (var.DF.branch.trk.truth.p.pdg == -13),
    (var.DF.branch.trk.truth.p.pdg == 211) | (var.DF.branch.trk.truth.p.pdg == -211),
    (var.DF.branch.trk.truth.p.pdg == 2212),
    (var.DF.branch.trk.truth.p.pdg == 22),
]
branchIDcategories.append(~branchIDcategories[0])
for c in branchIDcategories[1:-1]:
    branchIDcategories[-1] = branchIDcategories[-1] & ~c
    
branchIDcategories[0].name = "$\\mu$"
branchIDcategories[1].name = "$\\pi$"
branchIDcategories[2].name = "$p$"
branchIDcategories[3].name = "$\\gamma$"
branchIDcategories[4].name = "Other"

branchIDcategory_colors = trunkIDcategory_colors

In [ ]:
vtxIDcategories = [
    (trunkIDcategories[0] | trunkIDcategories[1]) & (branchIDcategories[0] | branchIDcategories[1]),
    ((trunkIDcategories[0] | trunkIDcategories[1]) & branchIDcategories[2]) |\
    (trunkIDcategories[2] & (branchIDcategories[0] | branchIDcategories[1])),
]

vtxIDcategories.append(~vtxIDcategories[0])
for c in branchIDcategories[1:-1]:
    vtxIDcategories[-1] = vtxIDcategories[-1] & ~c

vtxIDcategories[0].name = "$\\mu\\pi$, $\\pi\\pi$"
vtxIDcategories[1].name = "$\\mu p$, $\\pi p$"
vtxIDcategories[2].name = "Other"

vtxIDcategory_colors = [
    "#20634C",
    "#B13E29",
    "#67665B"
]


In [ ]:
def chi2(mc, onbeam, offbeam, scov):
    Ndata = onbeam.N - offbeam.N
    Nmc = mc.N
    stat_err = mc.N + offbeam.Nerr**2
    
    cov_select = np.ix_(np.where(mc.N != 0)[0], np.where(mc.N != 0)[0])
    cov = (scov + np.diag(stat_err))[cov_select]
    delta = (Nmc-Ndata)[Nmc != 0]
    
    return np.matmul(np.matmul(delta, np.linalg.inv(cov)), delta), int(np.sum(mc.N != 0))

In [ ]:
def mcdata_plot(v, bins, c, categories=None, xlabel=None, ylabel=None, 
                dosyst=True, systweights="all", varydataset=True, databinselect=None, color=None, fullpot=True, 
                ratio_lim=[0.5, 1.5], lgnd_ncol=3):
    if varydataset:
        syst_datasets = mc_systs
        syst_dataset_cut = cohlike
    else:
        syst_datasets = None
        syst_dataset_cut = None
        
    if databinselect is None:
        databinselect = np.zeros(bins.shape).astype(bool)
        databinselect[:] = True
                    
    hnom, hsyst = hist.makehist(v, mc, cut=c, bins=bins, POT=onbeam.POT if fullpot else onbeam_half.POT, categories=categories,
                systematics=dosyst, syst_weightname=systweights, 
                syst_datasets=syst_datasets, syst_dataset_cut=syst_dataset_cut)
    hcov = hist.makecovariance(sum(hnom), hsyst) if dosyst else np.diag(np.zeros((len(bins)-1,)))
    
    hcoh, _ = hist.makehist(v, mc, cut=c&cohlike, bins=bins, POT=onbeam.POT if fullpot else onbeam_half.POT)
    
    hon, _ = hist.makehist(v, onbeam if fullpot else onbeam_half, cut=c, bins=bins[databinselect], cvweight=False, systematics=False)
    hoff, _ = hist.makehist(v, offbeam, cut=c, bins=bins[databinselect], cvweight=False, 
                            livetime=onbeam.livetime if fullpot else onbeam_half.livetime, systematics=False)

    fig, (p0, p1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3,1]}, sharex=True, figsize=(7, 6.4))
    fig.subplots_adjust(hspace=0.03)

    hmc, hfill = hist.plotmc(p0, hnom, hcov, color=color)
    _, _, fh = hmc
    _,_, fcoh = p0.hist(hcoh.centers, bins=hcoh.bins, weights=hcoh.N, histtype="step", color="salmon", linewidth=4)
  
    hd = hist.plotdata(p0, hon-hoff)
    p0.set_ylim([p0.get_ylim()[0], p0.get_ylim()[1]*1.25])

    legend1 = p0.legend(handles=[hd, hfill] + fcoh, labels=["Data", "Syst. Err", "Coh-like MC"], 
                        loc="upper left", frameon=False) 
    
    # figure out how high to set legend
    if np.ceil(len(categories) / lgnd_ncol) == 1:
        ylgnd = 1.15
    elif np.ceil(len(categories) / lgnd_ncol) == 2:
        ylgnd = 1.25
    else:
        ylgnd = 1.3
    
    p0.legend(handles=fh, labels=[c.name for c in categories], 
                        loc="upper center", ncol=lgnd_ncol, bbox_to_anchor=(0.5, ylgnd))
    p0.add_artist(legend1)
    
    hcov_select = np.ix_(databinselect[:-1], databinselect[:-1])
    hmc_sum = sum(hnom)
    hmc_sum.N = hmc_sum.N[databinselect[:-1]]
    hmc_sum.Nerr = hmc_sum.Nerr[databinselect[:-1]]

    tchi2, ndof = chi2(hmc_sum, hon, hoff, hcov[hcov_select])

    p0.text(0.6, 0.9, "$\\chi^2/n$ = %.1f / %i" % (tchi2, ndof), fontsize=14,
            transform=p0.transAxes)
    
    hist.plotratio(plt, hon-hoff, hmc_sum, hcov[hcov_select])
    p1.axhline([1], color="r", linestyle=":")
    p1.set_ylim(ratio_lim)

    if xlabel:
        p1.set_xlabel(xlabel)
    if ylabel:
        p0.set_ylabel("%s / %s" % (ylabel, POTSTR if fullpot else POTSTR_half))
    p1.set_ylabel("Data / MC")
        
    return fig, (p0, p1)

In [ ]:
def recop(trk):
    p = trk.rangeP.p_muon
    p[~TrkInFV(trk.end)] = trk.mcsP.fwdP_muon
    return p

beam2det = np.array([ [0.921035925, 0.022715103, 0.388814672], [0, 0.998297825, -0.058321970], [-0.389477631, 0.053716629, 0.919468161]])

beamorigin = np.array([4.503730e2, 80.153901e2, 795.112945e2])

MUON_MASS = 0.1057 
PION_MASS = 0.13457

BEAMDIR = beam2det.dot(beamorigin) / np.linalg.norm(beam2det.dot(beamorigin)) 
BEAMDIR

In [ ]:
np.arccos(BEAMDIR[0]), np.arccos(BEAMDIR[0])*180/np.pi

In [ ]:
np.arccos(0)*180/np.pi

In [ ]:
cccoh = (np.abs(mccoh.df.slc.truth.pdg) == 14) & (mccoh.df.slc.truth.iscc == 1) & (mccoh.df.slc.truth.genie_mode == 3)

In [ ]:
mccoh.df.slc.truth[cccoh].p1

In [ ]:
offbeam.df["trunk_mcs_range"] = np.abs(1 - offbeam.df.trunk.trk.mcsP.fwdP_muon / offbeam.df.trunk.trk.rangeP.p_muon)
onbeam.df["trunk_mcs_range"] = np.abs(1 - onbeam.df.trunk.trk.mcsP.fwdP_muon / onbeam.df.trunk.trk.rangeP.p_muon)
onbeam_half.df["trunk_mcs_range"] = np.abs(1 - onbeam_half.df.trunk.trk.mcsP.fwdP_muon / onbeam_half.df.trunk.trk.rangeP.p_muon)
mc.df["trunk_mcs_range"] = np.abs(1 - mc.df.trunk.trk.mcsP.fwdP_muon / mc.df.trunk.trk.rangeP.p_muon)
for i in range(len(mc_systs_all)):
    mc_systs_all[i].df["trunk_mcs_range"] = np.abs(1 - mc_systs_all[i].df.trunk.trk.mcsP.fwdP_muon / mc_systs_all[i].df.trunk.trk.rangeP.p_muon)


offbeam.df["branch_mcs_range"] = np.abs(1 - offbeam.df.branch.trk.mcsP.fwdP_muon / offbeam.df.branch.trk.rangeP.p_muon)
onbeam.df["branch_mcs_range"] = np.abs(1 - onbeam.df.branch.trk.mcsP.fwdP_muon / onbeam.df.branch.trk.rangeP.p_muon)
onbeam_half.df["branch_mcs_range"] = np.abs(1 - onbeam_half.df.branch.trk.mcsP.fwdP_muon / onbeam_half.df.branch.trk.rangeP.p_muon)
mc.df["branch_mcs_range"] = np.abs(1 - mc.df.branch.trk.mcsP.fwdP_muon / mc.df.branch.trk.rangeP.p_muon)
for i in range(len(mc_systs_all)):
    mc_systs_all[i].df["branch_mcs_range"] = np.abs(1 - mc_systs_all[i].df.branch.trk.mcsP.fwdP_muon / mc_systs_all[i].df.branch.trk.rangeP.p_muon)
                                                                                                     

In [ ]:
offbeam.df["openangle"] = np.arccos(dotdf(offbeam.df.trunk.trk.dir, offbeam.df.branch.trk.dir))*180/np.pi
onbeam.df["openangle"] = np.arccos(dotdf(onbeam.df.trunk.trk.dir, onbeam.df.branch.trk.dir))*180/np.pi
onbeam_half.df["openangle"] = np.arccos(dotdf(onbeam_half.df.trunk.trk.dir, onbeam_half.df.branch.trk.dir))*180/np.pi
mc.df["openangle"] = np.arccos(dotdf(mc.df.trunk.trk.dir, mc.df.branch.trk.dir))*180/np.pi
for i in range(len(mc_systs_all)):
    mc_systs_all[i].df["openangle"] = np.arccos(dotdf(mc_systs_all[i].df.trunk.trk.dir, mc_systs_all[i].df.branch.trk.dir))*180/np.pi
               

In [ ]:
off_scalar_mom = offbeam.df.trunk.trk.dir.multiply(recop(offbeam.df.trunk.trk), axis=0) + \
           offbeam.df.branch.trk.dir.multiply(recop(offbeam.df.branch.trk), axis=0)
off_scalar_dir = off_scalar_mom.divide(magdf(off_scalar_mom), axis=0)
off_costhbeam =off_scalar_dir.x*BEAMDIR[0] + off_scalar_dir.y*BEAMDIR[1] + off_scalar_dir.z*BEAMDIR[2]
offbeam.df["beamangle"] = (180/np.pi)*np.arccos(off_costhbeam)
offbeam.df["pT"] = magdf(off_scalar_mom)*np.sqrt(1-off_costhbeam**2)

on_scalar_mom = onbeam.df.trunk.trk.dir.multiply(recop(onbeam.df.trunk.trk), axis=0) + \
           onbeam.df.branch.trk.dir.multiply(recop(onbeam.df.branch.trk), axis=0)
on_scalar_dir = on_scalar_mom.divide(magdf(on_scalar_mom), axis=0)
on_costhbeam = on_scalar_dir.x*BEAMDIR[0] + on_scalar_dir.y*BEAMDIR[1] + on_scalar_dir.z*BEAMDIR[2]
onbeam.df["beamangle"] = (180/np.pi)*np.arccos(on_costhbeam)
onbeam.df["pT"] = magdf(on_scalar_mom)*np.sqrt(1-on_costhbeam**2)

on_scalar_mom = onbeam_half.df.trunk.trk.dir.multiply(recop(onbeam_half.df.trunk.trk), axis=0) + \
           onbeam_half.df.branch.trk.dir.multiply(recop(onbeam_half.df.branch.trk), axis=0)
on_scalar_dir = on_scalar_mom.divide(magdf(on_scalar_mom), axis=0)
on_costhbeam = on_scalar_dir.x*BEAMDIR[0] + on_scalar_dir.y*BEAMDIR[1] + on_scalar_dir.z*BEAMDIR[2]
onbeam_half.df["beamangle"] = (180/np.pi)*np.arccos(on_costhbeam)
onbeam_half.df["pT"] = magdf(on_scalar_mom)*np.sqrt(1-on_costhbeam**2)

mc_scalar_mom = mc.df.trunk.trk.dir.multiply(recop(mc.df.trunk.trk), axis=0) + \
           mc.df.branch.trk.dir.multiply(recop(mc.df.branch.trk), axis=0)
mc_scalar_dir = mc_scalar_mom.divide(magdf(mc_scalar_mom), axis=0)
mc_costhbeam = mc_scalar_dir.x*BEAMDIR[0] + mc_scalar_dir.y*BEAMDIR[1] + mc_scalar_dir.z*BEAMDIR[2]
mc.df["beamangle"] = (180/np.pi)*np.arccos(mc_costhbeam)
mc.df["pT"] = magdf(mc_scalar_mom)*np.sqrt(1-mc_costhbeam**2)


for i in range(len(mc_systs_all)):
    mc_scalar_mom = mc_systs_all[i].df.trunk.trk.dir.multiply(recop(mc_systs_all[i].df.trunk.trk), axis=0) + \
               mc_systs_all[i].df.branch.trk.dir.multiply(recop(mc_systs_all[i].df.branch.trk), axis=0)
    mc_scalar_dir = mc_scalar_mom.divide(magdf(mc_scalar_mom), axis=0)
    mc_costhbeam = mc_scalar_dir.x*BEAMDIR[0] + mc_scalar_dir.y*BEAMDIR[1] + mc_scalar_dir.z*BEAMDIR[2]
    mc_systs_all[i].df["beamangle"] = (180/np.pi)*np.arccos(mc_costhbeam)
    mc_systs_all[i].df["pT"] = magdf(mc_scalar_mom)*np.sqrt(1-mc_costhbeam**2)

In [ ]:
def coht(df):
    trunkp = df.trunk.trk.dir.multiply(recop(df.trunk.trk), axis=0)
    branchp = df.branch.trk.dir.multiply(recop(df.branch.trk), axis=0)\
    
    branchp_larger = recop(df.trunk.trk) < recop(df.branch.trk)
    
    largerp = trunkp.copy()
    largerp[branchp_larger] = branchp[branchp_larger]
    
    smallerp = trunkp.copy()
    smallerp[~branchp_larger] = branchp[~branchp_larger]
    
    # assume muon is more energetic
    mu_p = largerp
    pi_p = smallerp*PION_MASS / MUON_MASS # scale to pion momentum
    
    mu_pL = mu_p.x*BEAMDIR[0] + mu_p.y*BEAMDIR[1] + mu_p.z*BEAMDIR[2]
    pi_pL = pi_p.x*BEAMDIR[0] + pi_p.y*BEAMDIR[1] + pi_p.z*BEAMDIR[2]
    
    mu_pT2 = magdf(mu_p)**2 - mu_pL**2
    pi_pT2 = magdf(pi_p)**2 - pi_pL**2
    
    mu_E = np.sqrt(magdf(mu_p)**2 + MUON_MASS**2)
    pi_E = np.sqrt(magdf(pi_p)**2 + PION_MASS**2)
                                             
    return mu_pT2 + pi_pT2 + (mu_E - mu_pL + pi_E - pi_pL)**2

In [ ]:
offbeam.df["coht"] = coht(offbeam.df)
onbeam.df["coht"] = coht(onbeam.df)
onbeam_half.df["coht"] = coht(onbeam_half.df)
mc.df["coht"] = coht(mc.df)
for i in range(len(mc_systs_all)):
    mc_systs_all[i].df["coht"] = coht(mc_systs_all[i].df)


In [ ]:
off_Etrunk = np.sqrt(MUON_MASS**2 + recop(offbeam.df.trunk.trk)**2)
off_Ebranch = np.sqrt(MUON_MASS**2 + recop(offbeam.df.branch.trk)**2)
offbeam.df["s_mass"] = np.sqrt(2)*np.sqrt(MUON_MASS**2 + off_Etrunk*off_Ebranch - recop(offbeam.df.trunk.trk)*recop(offbeam.df.branch.trk)*dotdf(offbeam.df.branch.trk.dir, offbeam.df.trunk.trk.dir))

on_Etrunk = np.sqrt(MUON_MASS**2 + recop(onbeam.df.trunk.trk)**2)
on_Ebranch = np.sqrt(MUON_MASS**2 + recop(onbeam.df.branch.trk)**2)
onbeam.df["s_mass"] = np.sqrt(2)*np.sqrt(MUON_MASS**2 + on_Etrunk*on_Ebranch - recop(onbeam.df.trunk.trk)*recop(onbeam.df.branch.trk)*dotdf(onbeam.df.branch.trk.dir, onbeam.df.trunk.trk.dir))

on_Etrunk = np.sqrt(MUON_MASS**2 + recop(onbeam_half.df.trunk.trk)**2)
on_Ebranch = np.sqrt(MUON_MASS**2 + recop(onbeam_half.df.branch.trk)**2)
onbeam_half.df["s_mass"] = np.sqrt(2)*np.sqrt(MUON_MASS**2 + on_Etrunk*on_Ebranch - recop(onbeam_half.df.trunk.trk)*recop(onbeam_half.df.branch.trk)*dotdf(onbeam_half.df.branch.trk.dir, onbeam_half.df.trunk.trk.dir))

mc_Etrunk = np.sqrt(MUON_MASS**2 + recop(mc.df.trunk.trk)**2)
mc_Ebranch = np.sqrt(MUON_MASS**2 + recop(mc.df.branch.trk)**2)
mc.df["s_mass"] = np.sqrt(2)*np.sqrt(MUON_MASS**2 + mc_Etrunk*mc_Ebranch - recop(mc.df.trunk.trk)*recop(mc.df.branch.trk)*dotdf(mc.df.branch.trk.dir, mc.df.trunk.trk.dir))

for i in range(len(mc_systs_all)):
    mc_Etrunk = np.sqrt(MUON_MASS**2 + recop(mc_systs_all[i].df.trunk.trk)**2)
    mc_Ebranch = np.sqrt(MUON_MASS**2 + recop(mc_systs_all[i].df.branch.trk)**2)
    mc_systs_all[i].df["s_mass"] = np.sqrt(2)*np.sqrt(MUON_MASS**2 + mc_Etrunk*mc_Ebranch - recop(mc_systs_all[i].df.trunk.trk)*recop(mc_systs_all[i].df.branch.trk)*dotdf(mc_systs_all[i].df.branch.trk.dir, mc_systs_all[i].df.trunk.trk.dir))


In [ ]:
onbeam.df[("trunk", "trk", "chi2pid", "I2", "chi2u", "")] = onbeam.df.trunk.trk.chi2pid.I2.chi2_muon
onbeam.df[("trunk", "trk", "chi2pid", "I2", "chi2p", "")] = onbeam.df.trunk.trk.chi2pid.I2.chi2_proton

onbeam.df[("branch", "trk", "chi2pid", "I2", "chi2u", "")] = onbeam.df.branch.trk.chi2pid.I2.chi2_muon
onbeam.df[("branch", "trk", "chi2pid", "I2", "chi2p", "")] = onbeam.df.branch.trk.chi2pid.I2.chi2_proton

onbeam_half.df[("trunk", "trk", "chi2pid", "I2", "chi2u", "")] = onbeam_half.df.trunk.trk.chi2pid.I2.chi2_muon
onbeam_half.df[("trunk", "trk", "chi2pid", "I2", "chi2p", "")] = onbeam_half.df.trunk.trk.chi2pid.I2.chi2_proton

onbeam_half.df[("branch", "trk", "chi2pid", "I2", "chi2u", "")] = onbeam_half.df.branch.trk.chi2pid.I2.chi2_muon
onbeam_half.df[("branch", "trk", "chi2pid", "I2", "chi2p", "")] = onbeam_half.df.branch.trk.chi2pid.I2.chi2_proton

offbeam.df[("trunk", "trk", "chi2pid", "I2", "chi2u", "")] = offbeam.df.trunk.trk.chi2pid.I2.chi2_muon
offbeam.df[("trunk", "trk", "chi2pid", "I2", "chi2p", "")] = offbeam.df.trunk.trk.chi2pid.I2.chi2_proton

offbeam.df[("branch", "trk", "chi2pid", "I2", "chi2u", "")] = offbeam.df.branch.trk.chi2pid.I2.chi2_muon
offbeam.df[("branch", "trk", "chi2pid", "I2", "chi2p", "")] = offbeam.df.branch.trk.chi2pid.I2.chi2_proton

In [ ]:
chi2u_trunk = variable.SystVariable(
    var.DF.trunk.trk.chi2pid.I2.chi2u, # CV
    variable.MultiVariable(
        var.DF.trunk.trk.chi2pid.I2_ghi.chi2u, # gain hi
        var.DF.trunk.trk.chi2pid.I2_glo.chi2u, # gain lo
    ),
    variable.MultiVariable(
        var.DF.trunk.trk.chi2pid.I2_calhi.chi2u, # dE/dx hi
        var.DF.trunk.trk.chi2pid.I2_callo.chi2u, # dE/dx lo

    ),
)

chi2u_branch = variable.SystVariable(
    var.DF.branch.trk.chi2pid.I2.chi2u, # CV
    variable.MultiVariable(
        var.DF.branch.trk.chi2pid.I2_ghi.chi2u, # gain hi
        var.DF.branch.trk.chi2pid.I2_glo.chi2u, # gain lo
    ),
    variable.MultiVariable(
        var.DF.branch.trk.chi2pid.I2_calhi.chi2u, # dE/dx hi
        var.DF.branch.trk.chi2pid.I2_callo.chi2u, # dE/dx lo
    ),
)

chi2p_trunk = variable.SystVariable(
    var.DF.trunk.trk.chi2pid.I2.chi2p, # CV
    variable.MultiVariable(
        var.DF.trunk.trk.chi2pid.I2_ghi.chi2p, # gain hi
        var.DF.trunk.trk.chi2pid.I2_glo.chi2p, # gain lo
    ),
    variable.MultiVariable(
        var.DF.trunk.trk.chi2pid.I2_calhi.chi2p, # dE/dx hi
        var.DF.trunk.trk.chi2pid.I2_callo.chi2p, # dE/dx lo

    ),
)

chi2p_branch = variable.SystVariable(
    var.DF.branch.trk.chi2pid.I2.chi2p, # CV
    variable.MultiVariable(
        var.DF.branch.trk.chi2pid.I2_ghi.chi2p, # gain hi
        var.DF.branch.trk.chi2pid.I2_glo.chi2p, # gain lo
    ),
    variable.MultiVariable(
        var.DF.branch.trk.chi2pid.I2_calhi.chi2p, # dE/dx hi
        var.DF.branch.trk.chi2pid.I2_callo.chi2p, # dE/dx lo
    ),
)

In [ ]:
def ratio_to_diff(r):
    return np.abs(1 - r)

trunk_mcs_range = variable.SystVariable(
        ratio_to_diff@(var.DF.trunk.trk.mcsP.fwdP_muon / var.DF.trunk.trk.rangeP.p_muon), # CV,
        variable.MultiVariable(
            ratio_to_diff@(var.DF.trunk.trk.mcsP_hi.fwdP_muon / var.DF.trunk.trk.rangeP.p_muon + 0.03), # MCS-hi
            ratio_to_diff@(var.DF.trunk.trk.mcsP_lo.fwdP_muon / var.DF.trunk.trk.rangeP.p_muon - 0.03), # MCS-lo
        ),
)

branch_mcs_range = variable.SystVariable(
        ratio_to_diff@(var.DF.branch.trk.mcsP.fwdP_muon / var.DF.branch.trk.rangeP.p_muon), # CV,
        variable.MultiVariable(
            ratio_to_diff@(var.DF.branch.trk.mcsP_hi.fwdP_muon / var.DF.branch.trk.rangeP.p_muon + 0.03), # MCS-hi
            ratio_to_diff@(var.DF.branch.trk.mcsP_lo.fwdP_muon / var.DF.branch.trk.rangeP.p_muon - 0.03), # MCS-lo
        ),
)

# for now, ignore systematic
trunk_mcs_range = trunk_mcs_range.cv()
branch_mcs_range = branch_mcs_range.cv()

In [ ]:
stubke_5mm = variable.SystVariable(
    var.DF.stub.l0_5cm.dedx*var.DF.stub.l0_5cm.length,
    variable.MultiVariable(
        var.DF.stub.l0_5cm.dedx_calhi*var.DF.stub.l0_5cm.length,
        var.DF.stub.l0_5cm.dedx_callo*var.DF.stub.l0_5cm.length,
    )
)

stubke_10mm = variable.SystVariable(
    var.DF.stub.l1cm.dedx*var.DF.stub.l1cm.length,
    variable.MultiVariable(
        var.DF.stub.l1cm.dedx_calhi*var.DF.stub.l1cm.length,
        var.DF.stub.l1cm.dedx_callo*var.DF.stub.l1cm.length,
    )
)

stubke_20mm = variable.SystVariable(
    var.DF.stub.l2cm.dedx*var.DF.stub.l2cm.length,
    variable.MultiVariable(
        var.DF.stub.l2cm.dedx_calhi*var.DF.stub.l2cm.length,
        var.DF.stub.l2cm.dedx_callo*var.DF.stub.l2cm.length,
    ),
)

stubke_30mm = variable.SystVariable(
    var.DF.stub.l3cm.dedx*var.DF.stub.l3cm.length,
    variable.MultiVariable(
        var.DF.stub.l3cm.dedx_calhi*var.DF.stub.l3cm.length,
        var.DF.stub.l3cm.dedx_callo*var.DF.stub.l3cm.length,
    ),
)

In [ ]:
othr_obj_cut = ~((var.DF.max_shw_len > 15) | (var.DF.max_othr_trk_len > 20) |\
                 (var.DF.min_othr_chi2_proton < 80) | (var.DF.max_othr_chi2_muon > 40))
othr_obj_cut = ~((var.DF.max_shw_len_primary > 10) | (var.DF.max_othr_trk_len > 10) |\
                 (var.DF.min_othr_chi2_proton < 90) | (var.DF.max_othr_chi2_muon > 75))

vloosetrunk_muid = (chi2u_trunk < 20).correlate() & (chi2p_trunk > 80) & (trunk_mcs_range < 0.5) & (var.DF.trunk.trk.len > 25)
vloosebranch_muid = (chi2u_branch < 20).correlate() & (chi2p_branch > 80) & (branch_mcs_range < 0.5) & (var.DF.branch.trk.len > 25)
vloosemuid = vloosetrunk_muid.correlate() & vloosebranch_muid

loosetrunk_muid = (chi2u_trunk < 15).correlate() & (chi2p_trunk > 80) & (trunk_mcs_range < 0.5) & (var.DF.trunk.trk.len > 25)
loosebranch_muid = (chi2u_branch < 15).correlate() & (chi2p_branch > 80) & (branch_mcs_range < 0.5) & (var.DF.branch.trk.len > 25)
loosemuid = loosetrunk_muid.correlate() & loosebranch_muid

tighttrunk_muid = (chi2u_trunk < 10).correlate() & (chi2p_trunk > 80) & (trunk_mcs_range < 0.5) & (var.DF.trunk.trk.len > 25)
tightbranch_muid = (chi2u_branch < 10).correlate() & (chi2p_branch > 80) & (branch_mcs_range < 0.5) & (var.DF.branch.trk.len > 25)

tighttrunk_muid = (chi2u_trunk < 10).correlate() & (chi2p_trunk > 95) & (trunk_mcs_range < 0.8) & (var.DF.trunk.trk.len > 25)
tightbranch_muid = (chi2u_branch < 10).correlate() & (chi2p_branch > 95) & (branch_mcs_range < 0.8) & (var.DF.branch.trk.len > 25)
tightmuid = tighttrunk_muid.correlate() & tightbranch_muid

one_tightmuid = tighttrunk_muid.correlate() | tightbranch_muid


hasstub = (stubke_5mm > 25) | (stubke_10mm > 30) |\
    (stubke_20mm > 35) | (stubke_30mm > 45)

stubid = ~hasstub


In [ ]:
c = loosemuid & cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.beamangle > 15)

c  = c.cv() #= c.fs[0]

In [ ]:
v = var.DF.openangle

bins = np.linspace(0, 180, 21)

mcdata_plot(v, bins, c, nucategories, "Opening Angle", "Slices", dosyst=True, color=nucategory_colors)

In [ ]:
c = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (np.abs@var.DF.trunk.trk.truth.p.pdg == 13)

bins = np.linspace(0, 20, 21)

v = var.DF.trunk.trk.chi2pid.I2.chi2u
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="CV")

v = var.DF.trunk.trk.chi2pid.I2_ghi.chi2u
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="Gain 1% Hi")

v = var.DF.trunk.trk.chi2pid.I2_glo.chi2u
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="Gain 1% Lo")

v = var.DF.trunk.trk.chi2pid.I2_callo.chi2u
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="dE/dx Low")

v = var.DF.trunk.trk.chi2pid.I2_calhi.chi2u
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="dE/dx High")

plt.legend(ncol=2, loc="upper center", bbox_to_anchor=(0.5, 1.3))

plt.xlabel("Trunk Track $\\chi^2_\\mu$")
plt.ylabel("Pre-Selected Events / %s" % POTSTR)

if dosave:
    plt.savefig(savedir + "trunk_chi2u_caldetvar.pdf", bbox_inches="tight")

In [ ]:
c = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (np.abs@var.DF.trunk.trk.truth.p.pdg == 13)

bins = np.linspace(80, 400, 21)

v = var.DF.trunk.trk.chi2pid.I2.chi2p
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="CV")

v = var.DF.trunk.trk.chi2pid.I2_ghi.chi2p
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="Gain 1% Hi")

v = var.DF.trunk.trk.chi2pid.I2_glo.chi2p
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="Gain 1% Lo")

v = var.DF.trunk.trk.chi2pid.I2_callo.chi2p
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="dE/dx Low")

v = var.DF.trunk.trk.chi2pid.I2_calhi.chi2p
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="dE/dx High")

plt.legend(ncol=2, loc="upper center", bbox_to_anchor=(0.5, 1.3))

plt.xlabel("Trunk Track $\\chi^2_p$")
plt.ylabel("Pre-Selected Events / %s" % POTSTR)

if dosave:
    plt.savefig(savedir + "trunk_chi2p_caldetvar.pdf", bbox_inches="tight")

In [ ]:
c = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.slc.truth.max_proton_ke < 0.05) & (var.DF.slc.truth.max_proton_ke > 0.015)

bins = np.linspace(0, 100, 21)

v = var.DF.stub.l0_5cm.dedx*var.DF.stub.l0_5cm.length
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="CV")

v = var.DF.stub.l0_5cm.dedx_calhi*var.DF.stub.l0_5cm.length
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="Recomb. High")

v = var.DF.stub.l0_5cm.dedx_callo*var.DF.stub.l0_5cm.length
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="Recomb. Low")

plt.xlabel("Stub KE [MeV]")
plt.title("Stubs (0 < Length < 5 mm)")
plt.legend()
plt.ylabel("Pre-Selected Events / %s" % POTSTR)

if dosave:
    plt.savefig(savedir + "stubke_5mm_caldetvar.pdf", bbox_inches="tight")

In [ ]:
c = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (np.abs@var.DF.trunk.trk.truth.p.pdg == 13)

bins = np.linspace(0, 1, 21)

v = ratio_to_diff@(var.DF.trunk.trk.mcsP.fwdP_muon / var.DF.trunk.trk.rangeP.p_muon)
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="CV")

v = ratio_to_diff@((var.DF.trunk.trk.mcsP.fwdP_muon + mc.df.trunk.trk.rangeP.p_muon*0.03) / var.DF.trunk.trk.rangeP.p_muon)
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="MCS-high")

v = ratio_to_diff@((var.DF.trunk.trk.mcsP.fwdP_muon - mc.df.trunk.trk.rangeP.p_muon*0.03) / var.DF.trunk.trk.rangeP.p_muon)
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="MCS-low")


plt.xlabel("|1 - MCS P / Range P|")
plt.legend()
plt.ylabel("Pre-Selected Events / %s" % POTSTR)

if dosave:
    plt.savefig(savedir + "trunkmcs_caldetvar.pdf", bbox_inches="tight")

In [ ]:
c = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.slc.truth.max_proton_ke < 0.05) & (var.DF.slc.truth.max_proton_ke > 0.015)

bins = np.linspace(0, 100, 21)

v = var.DF.stub.l1cm.dedx*var.DF.stub.l1cm.length
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="CV")

v = var.DF.stub.l1cm.dedx_calhi*var.DF.stub.l1cm.length
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="Recomb. High")

v = var.DF.stub.l1cm.dedx_callo*var.DF.stub.l1cm.length
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="Recomb. Low")

plt.xlabel("Stub KE [MeV]")
plt.title("Stubs (5 < Length < 10 mm)")
plt.legend()
plt.ylabel("Pre-Selected Events / %s" % POTSTR)

if dosave:
    plt.savefig(savedir + "stubke_10mm_caldetvar.pdf", bbox_inches="tight")

In [ ]:
c = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.slc.truth.max_proton_ke < 0.05) & (var.DF.slc.truth.max_proton_ke > 0.015)

bins = np.linspace(0, 150, 21)

v = var.DF.stub.l2cm.dedx*var.DF.stub.l2cm.length
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="CV")

v = var.DF.stub.l2cm.dedx_calhi*var.DF.stub.l2cm.length
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="Recomb. High")

v = var.DF.stub.l2cm.dedx_callo*var.DF.stub.l2cm.length
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="Recomb. Low")

plt.xlabel("Stub KE [MeV]")
plt.title("Stubs (10 < Length < 20 mm)")
plt.legend()
plt.ylabel("Pre-Selected Events / %s" % POTSTR)

if dosave:
    plt.savefig(savedir + "stubke_20mm_caldetvar.pdf", bbox_inches="tight")

In [ ]:
c = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.slc.truth.max_proton_ke < 0.05) & (var.DF.slc.truth.max_proton_ke > 0.015)

bins = np.linspace(0, 150, 21)

v = var.DF.stub.l3cm.dedx*var.DF.stub.l3cm.length
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="CV")

v = var.DF.stub.l3cm.dedx_calhi*var.DF.stub.l3cm.length
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="Recomb. High")

v = var.DF.stub.l3cm.dedx_callo*var.DF.stub.l3cm.length
_ = plt.hist(v(mc.df)[c(mc.df)], bins=bins, weights=mc.df.wgt.cv[c(mc.df)], histtype="step", label="Recomb. Low")

plt.xlabel("Stub KE [MeV]")
plt.title("Stubs (20 < Length < 30 mm)")
plt.legend()
plt.ylabel("Pre-Selected Events / %s" % POTSTR)

if dosave:
    plt.savefig(savedir + "stubke_30mm_caldetvar.pdf", bbox_inches="tight")

In [ ]:
trunk_mcs_range

In [ ]:
cuts = [
    cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.beamangle > 15),
    
    cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.beamangle > 15) &\
    (var.DF.openangle < 100) &\
    loosemuid,
    
    cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.beamangle > 15) &\
    (var.DF.openangle < 100) &\
    tightmuid,
    
    cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.beamangle > 15) &\
    (var.DF.openangle < 100) &\
    tightmuid & othr_obj_cut,
    
    cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.beamangle > 15) &\
    (var.DF.openangle < 100) &\
    tightmuid & othr_obj_cut & stubid,
    
    
]

In [ ]:
cut_text = [    
    "Pre-Selected",
    "Loose Muon ID",
    "Tight Muon ID",
    "Tight Muon ID\nOther Obj. Cut",
    "Tight Muon ID\nOther Obj. Cut\nStub ID",
]

In [ ]:
cut_dir = [    
    "PreSelection",
    "LooseMuonID",
    "TightMuonID",
    "TightMuonID_OtherObj",
    "TightMuonID_OtherObj_StubID",
]

In [ ]:
plotvars = [
    var.crlongtrkdiry@var.slc,
    var.DF.max_shw_len,
    var.DF.max_othr_trk_len,
    var.DF.max_othr_chi2_muon,
    var.DF.min_othr_chi2_proton,
    var.DF.trunk.trk.len,
    var.DF.branch.trk.len,
    var.DF.trunk.trackScore,
    var.DF.branch.trackScore,
    chi2u_trunk,
    chi2u_branch,
    chi2p_trunk,
    chi2p_branch,
    trunk_mcs_range,
    branch_mcs_range,
    stubke_5mm,
    stubke_10mm,
    stubke_20mm,
    stubke_30mm,
    var.DF.openangle,
    var.DF.beamangle,
    var.DF.s_mass,
    var.DF.coht,
    var.DF.pT,
]

In [ ]:
plotnames = [
    "CRLongTrkDirY",
    "Maximum Shower Length [cm]",
    "Maximum Other Track Length [cm]",
    "Maximum Other $\\chi^2_\\mu$",
    "Minimum Other $\\chi^2_p$",
    "Trunk Track Length [cm]",
    "Branch Track Length [cm]",
    "Trunk Track Score",
    "Branch Track Score",
    "Trunk $\\chi^2_\\mu$",
    "Branch $\\chi^2_\\mu$",
    "Trunk $\\chi^2_p$",
    "Branch $\\chi^2_p$",
    "Trunk (1 - MCS P / Range P)",
    "Branch (1 - MCS P / Range P)",
    "Stub KE (len < 0.5 cm) [MeV]",
    "Stub KE (0.5 < len < 1 cm) [MeV]",
    "Stub KE (1 < len < 2 cm) [MeV]",
    "Stub KE (2 < len < 3 cm) [MeV]",
    "Trunk - Branch Opening Angle [$^\\circ$]",
    "Scalar Angle to Beam [$^\\circ$]",
    "$M_{\\mu\\mu}$ [GeV/c$^2$]",
    "|t| [GeV$^2$]",
    "|Transverse Momentum| [GeV]",
]

In [ ]:
plotsave = [
    "crlongtrkdiry",
    "max_shw_len",
    "max_othr_trk_len",
    "max_othr_chi2u",
    "min_othr_chi2p",
    "trunk_trk_len",
    "branch_trk_len",
    "trunk_trk_score",
    "branch_trk_score",
    "trunk_chi2u",
    "branch_chi2u",
    "trunk_chi2p",
    "branch_chi2p",
    "trunk_mcsvrange",
    "branch_mcsvrange",
    "stub_ke_0_5cm",
    "stub_ke_1cm",
    "stub_ke_2cm",
    "stub_ke_3cm",
    "openangle",
    "beamangle",
    "s_mass",
    "coht",
    "pT",
]

In [ ]:
plotbins = [
    np.linspace(-1, 1, 21),
    np.linspace(0, 20, 21),
    np.linspace(0, 60, 21),
    np.linspace(0, 60, 21),
    np.linspace(0, 60, 21),
    np.linspace(0, 200, 21) + 10,
    np.linspace(0, 200, 21) + 10,
    np.linspace(0, 1, 21),
    np.linspace(0, 1, 21),
    np.linspace(0, 30, 16),
    np.linspace(0, 30, 16),
    np.linspace(80, 400, 21),
    np.linspace(80, 400, 21),
    np.linspace(0, 1, 21),
    np.linspace(0, 1, 21),
    np.linspace(0, 100, 6),
    np.linspace(0, 100, 6),
    np.linspace(0, 160, 9),
    np.linspace(0, 160, 9),
    np.linspace(0, 180, 21),
    np.linspace(0, 60, 13),
    np.linspace(0, 0.5, 21) + 2*MUON_MASS,
    np.linspace(0, 0.15, 16),
    np.linspace(0, 1, 11),
]

In [ ]:
len(list(zip(cuts, cut_text, cut_dir)))

In [ ]:
importlib.reload(hist)

In [ ]:
#  chi2pid
whichvars = [
    5, 7, 9, 11, 13
]

In [ ]:
ifig = 0
labels = ["CV", "Gain Hi", "Gain Lo", "dE/dx Hi", "dE/dx Lo"]

for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for ivar, (vs, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
        if ivar not in whichvars: continue
        if not isinstance(vs, variable.SystVariable): continue
            
        plt.figure(ifig)
        fig, (p0, p1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3,1]}, sharex=True, figsize=(7, 6.4))
        fig.subplots_adjust(hspace=0.03)
        
        hnom_cv = None
        isyst = 0
        for vv in vs.fs:
            if isinstance(vv, variable.MultiVariable):
                vvs = vv.fs
            else:
                vvs = [vv]
            for v in vvs:
                hnom, _ = hist.makehist(v, mc, cut=c, bins=bins, POT=onbeam.POT, systematics=False)
                if isyst == 0:
                    hnom_cv = hnom                
                _ = p0.hist(hnom.centers, bins=hnom.bins, weights=hnom.N, label=labels[isyst], histtype="step")
                isyst += 1
            
        hon, _ = hist.makehist(vs.cv(), onbeam, cut=c, bins=bins, cvweight=False, systematics=False)
        hoff, _ = hist.makehist(vs.cv(), offbeam, cut=c, bins=bins, cvweight=False, livetime=onbeam.livetime, systematics=False)     
        hd = hist.plotdata(p0, hon-hoff)
        p0.legend(loc="upper right")
            
        p0.text(0.9, 0.5, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        hist.plotratio(p1, hon-hoff, hnom_cv)
        p1.axhline([1], color="r", linestyle=":")
        p1.set_ylim([0.5, 1.5])
        
        p0.set_ylabel("Slices / %s" % POTSTR)
        p1.set_ylabel("Data / MC CV")
        p1.set_xlabel(name)

        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("nocat_%s.pdf" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("nocat_%s.svg" % savename)), bbox_inches="tight")
        
        ifig += 1
    if dosave: 
        plt.close("all")

In [ ]:
ifig = 0

for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for ivar, (v, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
        if ivar not in whichvars: continue
            
        plt.figure(ifig)

        fig, (p0, p1) = mcdata_plot(v, bins, c, trunkIDcategories, name, "Slices", dosyst=True, 
                                    lgnd_ncol=5, color=trunkIDcategory_colors)
        p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("trunkcat_%s.pdf" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("trunkcat_%s.svg" % savename)), bbox_inches="tight")
        
        ifig += 1        
    if dosave: 
        plt.close("all")

In [ ]:
# Branch chi2pid
whichvars = [
    6, 8, 10, 12, 14
]

In [ ]:
ifig = 0
labels = ["CV", "Gain Hi", "Gain Lo", "dE/dx Hi", "dE/dx Lo"]

for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for ivar, (vs, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
        if ivar not in whichvars: continue
        if not isinstance(vs, variable.SystVariable): continue
            
        plt.figure(ifig)
        fig, (p0, p1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3,1]}, sharex=True, figsize=(7, 6.4))
        fig.subplots_adjust(hspace=0.03)
        
        hnom_cv = None
        isyst = 0
        for vv in vs.fs:
            if isinstance(vv, variable.MultiVariable):
                vvs = vv.fs
            else:
                vvs = [vv]
            for v in vvs:
                hnom, _ = hist.makehist(v, mc, cut=c, bins=bins, POT=onbeam.POT, systematics=False)
                if isyst == 0:
                    hnom_cv = hnom                
                _ = p0.hist(hnom.centers, bins=hnom.bins, weights=hnom.N, label=labels[isyst], histtype="step")
                isyst += 1
            
        hon, _ = hist.makehist(vs.cv(), onbeam, cut=c, bins=bins, cvweight=False, systematics=False)
        hoff, _ = hist.makehist(vs.cv(), offbeam, cut=c, bins=bins, cvweight=False, livetime=onbeam.livetime, systematics=False)     
        hd = hist.plotdata(p0, hon-hoff)
        p0.legend(loc="upper right")
            
        p0.text(0.9, 0.5, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        hist.plotratio(p1, hon-hoff, hnom_cv)
        p1.axhline([1], color="r", linestyle=":")
        p1.set_ylim([0.5, 1.5])
        
        p0.set_ylabel("Slices / %s" % POTSTR)
        p1.set_ylabel("Data / MC CV")
        p1.set_xlabel(name)

        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("nocat_%s.pdf" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("nocat_%s.svg" % savename)), bbox_inches="tight")
        
        ifig += 1
    if dosave: 
        plt.close("all")

In [ ]:
ifig = 0

for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for ivar, (v, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
        if ivar not in whichvars: continue
            
        plt.figure(ifig)

        fig, (p0, p1) = mcdata_plot(v, bins, c, branchIDcategories, name, "Slices",
                                    lgnd_ncol=5, color=branchIDcategory_colors)
        p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("branchcat_%s.pdf" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("branchcat_%s.svg" % savename)), bbox_inches="tight")
        
        ifig += 1        
    if dosave: 
        plt.close("all")

In [ ]:
# stubs
whichvars = [
    15, 16, 17, 18
]

In [ ]:
ifig = 0
labels = ["CV", "Recomb-Hi", "Recomb-Lo"]

for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for ivar, (vs, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
        if ivar not in whichvars: continue
        plt.figure(ifig)
        fig, (p0, p1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3,1]}, sharex=True, figsize=(7, 6.4))
        fig.subplots_adjust(hspace=0.03)
        
        hnom_cv = None
        isyst = 0
        for vv in vs.fs:
            if isinstance(vv, variable.MultiVariable):
                vvs = vv.fs
            else:
                vvs = [vv]
            for v in vvs:
                hnom, _ = hist.makehist(v, mc, cut=c, bins=bins, POT=onbeam.POT, systematics=False)
                if isyst == 0:
                    hnom_cv = hnom                
                _ = p0.hist(hnom.centers, bins=hnom.bins, weights=hnom.N, label=labels[isyst], histtype="step")
                isyst += 1
            
        hon, _ = hist.makehist(vs.cv(), onbeam, cut=c, bins=bins, cvweight=False, systematics=False)
        hoff, _ = hist.makehist(vs.cv(), offbeam, cut=c, bins=bins, cvweight=False, livetime=onbeam.livetime, systematics=False)     
        hd = hist.plotdata(p0, hon-hoff)
        p0.legend(loc="upper right")
            
        p0.text(0.9, 0.5, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        hist.plotratio(p1, hon-hoff, hnom_cv)
        p1.axhline([1], color="r", linestyle=":")
        p1.set_ylim([0.5, 1.5])
        
        p0.set_ylabel("Slices / %s" % POTSTR)
        p1.set_ylabel("Data / MC CV")
        p1.set_xlabel(name)

        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("nocat_%s.pdf" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("nocat_%s.svg" % savename)), bbox_inches="tight")
        
        ifig += 1
    if dosave: 
        plt.close("all")

In [ ]:
ifig = 0

for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for ivar, (v, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
        if ivar not in whichvars: continue
            
        plt.figure(ifig)

        fig, (p0, p1) = mcdata_plot(v, bins, c, vtxIDcategories, name, "Slices", color=vtxIDcategory_colors)
        p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("vtxcat_%s.pdf" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("vtxcat_%s.svg" % savename)), bbox_inches="tight")
        
        ifig += 1        
#     if dosave: 
#         plt.close("all")

In [ ]:
whichvars = list(range(len(plotvars))) # all

In [ ]:
ifig = 0

for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for ivar, (v, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
        if ivar not in whichvars: continue
            
        plt.figure(ifig)

        fig, (p0, p1) = mcdata_plot(v, bins, c, nucategories, name, "Slices", color=nucategory_colors)
        p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("%s.pdf" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("%s.svg" % savename)), bbox_inches="tight")
        
        ifig += 1        
    if dosave: 
        plt.close("all")

In [ ]:
# open angle plot

In [ ]:
ctxt = "Loose Muon ID"
cdir = "LooseMuID"
print(ctxt)

if dosave:
    os.makedirs(os.path.join(savedir, cdir), exist_ok=True)

In [ ]:
c = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.openangle < 100) &\
    loosemuid

v = var.DF.beamangle
bins = np.linspace(0, 60, 13)
name = "Scalar Angle to Beam [$^\\circ$]"

In [ ]:
fig, (p0, p1) = mcdata_plot(v, bins, c, nucategories, name, "Slices", 
                            databinselect=(bins >= 15), color=nucategory_colors, fullpot=True, ratio_lim=[0, 2], 
                            dosyst=True)
p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")

if dosave: 
    plt.savefig(os.path.join(savedir, cdir, "thnumi_15deg_fullpot.pdf"), bbox_inches="tight")
    plt.savefig(os.path.join(savedir, cdir, "thnumi_15deg_fullpot.svg"), bbox_inches="tight")

In [ ]:
ctxt = "Tight Muon ID\nOther Obj. Cut\nStub ID"
cdir = "NearSideband"
print(ctxt)

if dosave:
    os.makedirs(os.path.join(savedir, cdir), exist_ok=True)

In [ ]:
c = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.openangle < 60) &\
    tightmuid & othr_obj_cut & stubid

v = var.DF.beamangle
bins = np.linspace(0, 60, 13)
name = "Scalar Angle to Beam [$^\\circ$]"

In [ ]:
fig, (p0, p1) = mcdata_plot(v, bins, c, nucategories, name, "Slices", 
                            databinselect=(bins >= 15), color=nucategory_colors, fullpot=True, ratio_lim=[0, 2], 
                            dosyst=True)
p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")

if dosave: 
    plt.savefig(os.path.join(savedir, cdir, "thnumi_15deg_fullpot.pdf"), bbox_inches="tight")
    plt.savefig(os.path.join(savedir, cdir, "thnumi_15deg_fullpot.svg"), bbox_inches="tight")

In [ ]:
fig, (p0, p1) = mcdata_plot(v, bins, c, nucategories, name, "Slices", 
                            databinselect=(bins >= 15), color=nucategory_colors, fullpot=False, ratio_lim=[0, 2])
p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")

if dosave: 
    plt.savefig(os.path.join(savedir, cdir, "thnumi_15deg_halfpot.pdf"), bbox_inches="tight")
    plt.savefig(os.path.join(savedir, cdir, "thnumi_15deg_halfpot.svg"), bbox_inches="tight")

In [ ]:
fig, (p0, p1) = mcdata_plot(v, bins, c, nucategories, name, "Slices", 
                            databinselect=(bins >= 5), color=nucategory_colors, fullpot=False, ratio_lim=[0, 2])
p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")

if dosave: 
    plt.savefig(os.path.join(savedir, cdir, "thnumi_5deg_halfpot.pdf"), bbox_inches="tight")
    plt.savefig(os.path.join(savedir, cdir, "thnumi_5deg_halfpot.svg"), bbox_inches="tight")

In [ ]:
c = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.openangle < 60) &\
    (var.DF.beamangle > 5) & (var.DF.beamangle < 10) &\
    tightmuid & othr_obj_cut & stubid

ctxt = "Near Sideband"

# kinematics
whichvars = list(range(len(plotvars)))[-5:]

# redo binning
nearsideband_plotbins = [
    np.linspace(0, 60, 7),
    np.linspace(0, 60, 13),
    np.linspace(0, 0.5, 6) + 2*MUON_MASS,
    np.linspace(0, 0.2, 6),
    np.linspace(0, 0.6, 7),
]

In [ ]:
ifig = 0


for ivar, (v, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
    if ivar not in whichvars: continue

    plt.figure(ifig)

    fig, (p0, p1) = mcdata_plot(v, nearsideband_plotbins[ifig], c, nucategories, name, "Slices", 
                                color=nucategory_colors, fullpot=False, ratio_lim=[0, 2])
    p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")

    if dosave: 
        plt.savefig(os.path.join(savedir, cdir, ("nearsideband_%s.pdf" % savename)), bbox_inches="tight")
        plt.savefig(os.path.join(savedir, cdir, ("nearsideband_%s.svg" % savename)), bbox_inches="tight")

    ifig += 1        
# if dosave: 
#     plt.close("all")

In [ ]:
c = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.openangle < 60) &\
    (var.DF.beamangle > 5) & (var.DF.beamangle < 10) &\
    tightmuid & othr_obj_cut & stubid

numu_cc_coh = (var.DF.slc.truth.genie_mode == 3) & ((np.abs@var.DF.slc.truth.pdg) == 14) & (var.DF.slc.truth.iscc == 1)

@var.VAR
def one(df):
    return pd.Series(1, df.index)

v = one
bins = np.linspace(0, 2, 2)

In [ ]:
hnom_cccoh, hsyst_cccoh = hist.makehist(v, mc, cut=c&numu_cc_coh, bins=bins, POT=onbeam.POT,
            systematics=True, syst_weightname="all", 
            syst_datasets=mc_systs, syst_dataset_cut=cohlike)

hnom_bkg, hsyst_bkg = hist.makehist(v, mc, cut=c&(~numu_cc_coh), bins=bins, POT=onbeam.POT,
            systematics=True, syst_weightname="xsec")

cccoh_err = hist.makecovariance(hnom_cccoh, hsyst_cccoh).flatten()
bkg_err = hist.makecovariance(hnom_bkg, hsyst_bkg).flatten()

hon, _ = hist.makehist(v, onbeam_half, cut=c, bins=bins, cvweight=False, systematics=False)
hoff, _ = hist.makehist(v, offbeam, cut=c, bins=bins, cvweight=False, 
                        livetime=onbeam_half.livetime, systematics=False)

In [ ]:
Nexp_evt = hnom_cccoh.N
Nexp_bkg_evt = hnom_bkg.N
Nexp_err = np.sqrt(Nexp_evt + Nexp_bkg_evt + bkg_err)
Nexp_halfpot_err = np.sqrt((Nexp_evt + Nexp_bkg_evt)/2 + bkg_err/4)

In [ ]:
Nobs_evt = hon.N - hnom_bkg.N/2
Nobs_err = np.sqrt(hon.N + bkg_err/4)

In [ ]:
def plot_chi2(xs, cv, err):
    return (xs-cv)**2 /(2*err**2)

xs = np.linspace(-2, 4, 101)

plt.plot(xs, plot_chi2(xs, 1, np.sqrt(cccoh_cov)/Nexp_evt), label="Prior")

plt.plot(xs, plot_chi2(xs, Nobs_evt/(Nexp_evt/2), Nobs_err/(Nexp_evt/2)), 
         color="black", linestyle=":",
         label="Obs. Constraint, Half POT")

l1 = plt.plot(xs, plot_chi2(xs, 1, Nexp_err/Nexp_evt), 
         label="Exp. Constraint, Full POT")

plt.plot(xs, plot_chi2(xs, 1, Nexp_halfpot_err/(Nexp_evt/2)), 
         color=l1[0].get_color(), linestyle=":",
         label="Exp. Constraint, Half POT")

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.225), ncol=2)
plt.xlabel("$\\nu_\\mu$ CC Coherent-$\\pi$ Strength")
plt.ylabel("$\\Delta \\chi^2$")
plt.ylim([0, 2.4])
plt.grid()

In [ ]:
hnom, hsyst = hist.makehist(v, mc, cut=c, bins=bins, POT=onbeam.POT,
            systematics=True, syst_weightname="all", 
            syst_datasets=mc_systs, syst_dataset_cut=cohlike)
hcov = hist.makecovariance(hnom, hsyst)

hon, _ = hist.makehist(v, onbeam_half, cut=c, bins=bins, cvweight=False, systematics=False)
hoff, _ = hist.makehist(v, offbeam, cut=c, bins=bins, cvweight=False, 
                        livetime=onbeam_half.livetime, systematics=False)

In [ ]:
c = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.openangle < 60) &\
    (var.DF.beamangle > 5) & (var.DF.beamangle < 10) &\
    tightmuid & othr_obj_cut & stubid

whichevt = pd.Series(False, onbeam_half.hdrdf.index) | c(onbeam_half.df).groupby(level=[0,1]).any()

In [ ]:
["%i:%i" % (r[1]["run"], r[1]["evt"]) for r in onbeam.hdrdf.loc[whichevt].iterrows()]

In [ ]:
FILELIST = "/icarus/app/users/gputnam/DP2023G/ICARUSRun2DataOnBeamMajority_stage0_disk_files_deduped.list"
NFILE_PER_JOB = 5

with open(FILELIST) as f:
    FILES = [l.rstrip("\n") for l in f]
FILES

In [ ]:
PROCS = [r[1]["proc"].values[0] for r in onbeam.hdrdf.loc[whichevt].iterrows()]
files_to_process = [FILES[P*NFILE_PER_JOB:(P+1)*NFILE_PER_JOB] for P in PROCS]
files_to_process = [f for files in files_to_process for f in files]
len(files_to_process)

In [ ]:
with open("/exp/icarus/app/users/gputnam/DP2023G/INTERESTING_FILES.list", "w+") as f:
    for fname in files_to_process:
        f.write(fname + "\n")

In [ ]:
FILELIST = "/exp/icarus/app/users/gputnam/DP2023G/localProducts_larsoft_v09_72_00_e20_prof/icaruscode/ICARUSRun1OnBeamMajority_2022A_files.list"
NFILE_PER_JOB = 5

with open(FILELIST) as f:
    FILES = [l.rstrip("\n") for l in f]
FILES

In [ ]:
PROCS = [r[1]["proc"].values[0] for r in onbeam.hdrdf.loc[whichevt].iterrows()]
files_to_process = [FILES[P*NFILE_PER_JOB:(P+1)*NFILE_PER_JOB] for P in PROCS]
files_to_process = [f for files in files_to_process for f in files]
len(files_to_process)

In [ ]:
with open("/exp/icarus/app/users/gputnam/DP2023G/INTERESTING_FILES_Run1.list", "w+") as f:
    for fname in files_to_process:
        f.write(fname + "\n")

In [ ]:
branches = [
    ("slc", "vertex", "x", "", "", ""),
    ("slc", "vertex", "y", "", "", ""),
    ("slc", "vertex", "z", "", "", ""),
    ("trunk", "trk", "len", "", "", ""),
    ("trunk", "trk", "chi2pid", "I2", "chi2_muon", ""),
    ("trunk", "trk", "chi2pid", "I2", "chi2_proton", ""),
    ("branch", "trk", "len", "", "", ""),
    ("branch", "trk", "chi2pid", "I2", "chi2_muon", ""),
    ("branch", "trk", "chi2pid", "I2", "chi2_proton", ""),
]

In [ ]:
onbeam_half.df.loc[c(onbeam_half.df), branches]

In [ ]:
onbeam_half.df.trunk.trk.len

In [ ]:
stdcols = mc.df.wgt.coh.columns[1:]
stdcols

In [ ]:
_ = plt.hist(mc.df.wgt.coh[(mc.df.slc.truth.genie_mode == 3) & c(mc.df)][stdcols].std(1))

In [ ]:
np.sqrt((mc.df.wgt.coh[(mc.df.slc.truth.genie_mode == 3) & c(mc.df)][stdcols].std(1)**2).mean())

In [ ]:
# Sideband study

In [ ]:
ifig = 0

for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for ivar, (v, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
        if ivar not in whichvars: continue
            
        plt.figure(ifig)

        v = v if not isinstance(v, variable.MultiVariable) else v.cv()
        hnom,_ = hist.makehist(v, mc, cut=c, bins=bins, POT=onbeam.POT, categories=nucategories, systematics=False)          
        _ = plt.hist([h.centers for h in hnom], 
                    bins=hnom[0].bins, weights=[h.N for h in hnom], 
                    label=[cat.name for cat in nucategories], stacked=True)
        plt.legend(loc="upper center", ncol=3, bbox_to_anchor=(0.5, 1.35))
        plt.xlabel(name)
        plt.ylabel("Events / %s" % POTSTR)
        plt.text(0.95, 0.95, ctxt, transform=plt.gca().transAxes, verticalalignment='top', horizontalalignment="right")

        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("mccv_%s.pdf" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("mccv_%s.svg" % savename)), bbox_inches="tight")
        
        ifig += 1 
            
#     if dosave: 
#         plt.close("all")

In [ ]:
whichvars

In [ ]:
othr_obj_cut = ~((var.DF.max_shw_len_primary > 15) | (var.DF.max_othr_trk_len > 20) |\
                 (var.DF.min_othr_chi2_proton_all < 80))

trunk_muid = (var.DF.trunk.trk.len > 25) & (var.DF.trunk.trk.chi2pid.I2.chi2_muon < 10) &\
    (var.DF.trunk.trk.chi2pid.I2.chi2_proton > 80) & (var.DF.trunk_mcs_range < 0.5)
branch_muid = (var.DF.branch.trk.len > 25) & (var.DF.branch.trk.chi2pid.I2.chi2_muon < 10) &\
    (var.DF.branch.trk.chi2pid.I2.chi2_proton > 80) & (var.DF.branch_mcs_range < 0.5)

hasstub = (var.DF.stub.l0_5cm.dqdx > 5e5) | (var.DF.stub.l1cm.dqdx > 3e5) |\
    (var.DF.stub.l2cm.dqdx > 2.5e5) | (var.DF.stub.l3cm.dqdx > 2e5)

stubid = ~hasstub

In [ ]:
othr_obj_cut = ~((var.DF.max_shw_len_primary > 10) | (var.DF.max_othr_trk_len > 20) |\
                 (var.DF.min_othr_chi2_proton < 80) | (var.DF.max_othr_chi2_muon > 40))

# othr_obj_cut = ~((var.DF.max_shw_len > 15) | (var.DF.max_othr_trk_len > 20) |\
#                  (var.DF.min_othr_chi2_proton_all < 80) | (var.DF.max_othr_chi2_muon > 40))

trunk_muid = (var.DF.trunk.trk.len > 25) & (var.DF.trunk.trk.chi2pid.I2.chi2_muon < 10) &\
    (var.DF.trunk.trk.chi2pid.I2.chi2_proton > 80) & (var.DF.trunk_mcs_range < 0.5)
branch_muid = (var.DF.branch.trk.len > 25) & (var.DF.branch.trk.chi2pid.I2.chi2_muon < 10) &\
    (var.DF.branch.trk.chi2pid.I2.chi2_proton > 80) & (var.DF.branch_mcs_range < 0.5)

In [ ]:
nmu = mc.df.slc.truth.iscc & (np.abs(mc.df.slc.truth.pdg) == 14)
ns = (np.abs(mc.df.slc.truth.p0.pdg) == 3112) + (np.abs(mc.df.slc.truth.p0.pdg) == 3222) +\
      (np.abs(mc.df.slc.truth.p1.pdg) == 3112) + (np.abs(mc.df.slc.truth.p1.pdg) == 3222)
is_coh_like = (mc.df.slc.truth.max_proton_ke < 0.05) & (mc.df.slc.truth.npi + nmu + ns >= 2)

In [ ]:
when = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.openangle < 0.5*180/np.pi) &\
    trunk_muid & branch_muid & othr_obj_cut

In [ ]:
c = when(mc.df)

In [ ]:
(c & is_coh_like).sum() / c.sum()

In [ ]:
(2*onbeam.POT / mc.POT)*mc.df.wgt.cv[c].sum()

In [ ]:
2*onbeam.POT

In [ ]:
v = var.DF.beamangle(mc.df)
whens = [c & is_coh_like, c & ~is_coh_like]
labels = ["Coh-Like", "Not-Coh-Like"]

_ = plt.hist([v[w] for w in whens], label=labels, stacked=True, bins=np.linspace(0, 75, 11))
plt.legend()
plt.xlabel("$\\theta_\\mathrm{NuMI}$")
plt.ylabel("Events / Data POT")
plt.title("Selected Di-Muon Decay Candidates")
plt.axvline(7.5, color="red", linestyle=":", linewidth=2)
plt.text(0.01, 0.8, "Signal\nRegion", color="red", transform=plt.gca().transAxes, fontsize=12)

In [ ]:
for i, col in enumerate(cols):
    plt.figure(i)
    
    N,bins,_ = plt.hist(mc.df[c & ~is_coh_like][col], histtype="step")
    plt.hist(mc.df[c & is_coh_like][col], bins=bins, histtype="step")
    plt.xlabel(col)

In [ ]:
mc.df[(c & ~is_coh_like)].slc.truth.p0

In [ ]:
mc.df[(c & ~is_coh_like)].slc.truth.p1

In [ ]:
mc.df[(c & ~is_coh_like)].trunk.trk.truth

In [ ]:
mc.df[(c & ~is_coh_like)].branch.trk.truth

In [ ]:
mc.df[(c & ~is_coh_like) & (mc.df.branch.trk.truth.p.pdg == 2212)].branch.trk.chi2pid

In [ ]:
["%i:%i" % (r[1]["run"], r[1]["evt"]) for r in onbeam.hdrdf.loc[whichevt].iterrows()]

In [ ]:
FILELIST = "/icarus/app/users/gputnam/DP2023G/ICARUSRun2DataOnBeamMajority_stage0_disk_files_deduped.list"
NFILE_PER_JOB = 5

with open(FILELIST) as f:
    FILES = [l.rstrip("\n") for l in f]
FILES

In [ ]:
PROCS = [r[1]["proc"].values[0] for r in onbeam.hdrdf.loc[whichevt].iterrows()]
files_to_process = [FILES[P*NFILE_PER_JOB:(P+1)*NFILE_PER_JOB] for P in PROCS]
files_to_process = [f for files in files_to_process for f in files]
len(files_to_process)

In [ ]:
with open("/icarus/app/users/gputnam/DP2023G/INTERESTING_FILES.list", "w+") as f:
    for fname in files_to_process:
        f.write(fname + "\n")

In [ ]:
[r for r in onbeam.hdrdf.loc[whichevt].iterrows()][0][1]

In [ ]:
# Run 1 only test

In [ ]:
ifig = 0

onbeam_tmp = onbeam
offbeam_tmp = offbeam

onbeam = Dataset(onbeam.df[onbeam.df.Run1], onbeamR1.livetime, onbeamR1.POT)
offbeam = Dataset(offbeam.df[offbeam.df.Run1], offbeamR1.livetime, offbeamR1.POT)

for c, ctxt, cdir in zip(cuts[:3], cut_text[:3], cut_dir[:3]):
    ctxt = ctxt + "\nRun 1"
    cdir = cdir + "_Run1"
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for v, name, savename, bins in zip(plotvars, plotnames, plotsave, plotbins):
        plt.figure(ifig)

        fig, (p0, p1) = mcdata_plot(v, bins, c, nucategories, name, "Slices")
        p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("%s.svg" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("%s.pdf" % savename)), bbox_inches="tight")
        
        ifig += 1        
    if dosave: 
        plt.close("all")

In [ ]:
onbeam = onbeam_tmp
offbeam = offbeam_tmp

In [ ]:
# Run 2 only test

In [ ]:
ifig = 0

onbeam = Dataset(onbeam.df[onbeam.df.Run2], onbeamR2.livetime, onbeamR2.POT)
offbeam = Dataset(offbeam.df[offbeam.df.Run2], offbeamR2.livetime, offbeamR2.POT)

for c, ctxt, cdir in zip(cuts[:3], cut_text[:3], cut_dir[:3]):
    ctxt = ctxt + "\nRun 2"
    cdir = cdir + "_Run2"
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for v, name, savename, bins in zip(plotvars, plotnames, plotsave, plotbins):
        plt.figure(ifig)

        fig, (p0, p1) = mcdata_plot(v, bins, c, nucategories, name, "Slices")
        p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("%s.pdf" % savename)), bbox_inches="tight")
        
        ifig += 1        
    if dosave: 
        plt.close("all")

In [ ]:
onbeam = onbeam_tmp
offbeam = offbeam_tmp